## How do you simulate dependent dice rolls in anydice?

[StackExchange.](https://rpg.stackexchange.com/questions/197916/how-do-you-simulate-dependent-dice-rolls-in-anydice)

This is the following scenario in 5e D&D:

1. First, the player casts *mind sliver*. Target must make an Int save or suffer 2d6 damage, and a -1d4 penalty to their next saving throw.
2. Then the player sends two *minute meteors* against the target, who suffers 2d6 damage with a Dex save for half.

In [1]:
import piplite
await piplite.install("icepool")

import icepool

from icepool import d

dc = 15
dex_save_mod = 1
int_save_mod = 1
mind_sliver_damage = 2 @ d(6)
meteor_damage = 2 @ d(6)

In [2]:
# Style 1: using direct expression.

# First, express the damage result of a meteor as a die.
meteor = meteor_damage // (1 + (dex_save_mod + d(20) >= dc))
sliver_meteor = meteor_damage // (1 + (dex_save_mod + d(20) - d(4) >= dc))

# Then, choose a sum based on the result of the save against *mind sliver*.
net_damage = (int_save_mod + d(20) < dc).if_else(
    mind_sliver_damage + sliver_meteor + meteor,
    meteor + meteor
)

print(net_damage)
print('Mean: ', net_damage.mean())

Denominator: 1492992000

| Outcome |   Weight | Probability |
|--------:|---------:|------------:|
|       2 |   444528 |   0.029774% |
|       3 |  2624832 |   0.175810% |
|       4 |  8250102 |   0.552588% |
|       5 | 17316780 |   1.159871% |
|       6 | 27671950 |   1.853456% |
|       7 | 36747456 |   2.461330% |
|       8 | 44712588 |   2.994831% |
|       9 | 52931776 |   3.545349% |
|      10 | 62124420 |   4.161069% |
|      11 | 71072532 |   4.760409% |
|      12 | 79249786 |   5.308119% |
|      13 | 86235552 |   5.776022% |
|      14 | 91764738 |   6.146365% |
|      15 | 95246840 |   6.379595% |
|      16 | 96978180 |   6.495559% |
|      17 | 97277856 |   6.515631% |
|      18 | 95569162 |   6.401184% |
|      19 | 91354536 |   6.118890% |
|      20 | 84550284 |   5.663144% |
|      21 | 75907104 |   5.084227% |
|      22 | 65890890 |   4.413345% |
|      23 | 55201224 |   3.697356% |
|      24 | 44606276 |   2.987710% |
|      25 | 34718736 |   2.325447% |
|      26 | 2

In [3]:
# Style 2: using state transitions.

# Define state transition functions.
# The state consists of the total damage dealt so far and whether the target has the *mind sliver* debuff.
# These functions take a state and update them with the results of one event.

def mind_sliver(damage, debuff):
    return (int_save_mod - debuff @ d(4) + d(20) < dc).if_else(
        (damage + mind_sliver_damage, True),
        (damage, False)
    )

def one_meteor(damage, debuff):
    return (dex_save_mod - debuff @ d(4) + d(20) < dc).if_else(
        (damage + meteor_damage, False),
        (damage + meteor_damage // 2, False)
    )

# Then, apply them to the state using the sub() method.
# The initial state is 0 damage dealt so far and no *mind sliver* debuff.
state = icepool.Die([(0, False)])
# The star option unpacks outcomes before giving them to the function.
state = state.sub(mind_sliver, star=1)
state = state.sub(one_meteor, star=1)
state = state.sub(one_meteor, star=1)

print(state)
print('Mean: ', state[0].mean())

Denominator: 1492992000

| Outcome[0] | Outcome[1] |   Weight | Probability |
|-----------:|-----------:|---------:|------------:|
|          2 |      False |   444528 |   0.029774% |
|          3 |      False |  2624832 |   0.175810% |
|          4 |      False |  8250102 |   0.552588% |
|          5 |      False | 17316780 |   1.159871% |
|          6 |      False | 27671950 |   1.853456% |
|          7 |      False | 36747456 |   2.461330% |
|          8 |      False | 44712588 |   2.994831% |
|          9 |      False | 52931776 |   3.545349% |
|         10 |      False | 62124420 |   4.161069% |
|         11 |      False | 71072532 |   4.760409% |
|         12 |      False | 79249786 |   5.308119% |
|         13 |      False | 86235552 |   5.776022% |
|         14 |      False | 91764738 |   6.146365% |
|         15 |      False | 95246840 |   6.379595% |
|         16 |      False | 96978180 |   6.495559% |
|         17 |      False | 97277856 |   6.515631% |
|         18 |      F